## Usage
Make sure azure_notebook_reporting is installed as a first step, run below using a code cell once off

```python
%pip install --quiet git+https://github.com/wagov/wasoc-notebook.git
```

To run in Azure ML
```python
import os, json, pandas as pd
from azure_notebook_reporting import KQL, BlobPath
os.environ.update(json.load(open(f"{os.environ['HOME']}/cloudfiles/code/nbenv.json")))
path = BlobPath(os.environ["AZURE_STORAGE_CONTAINER"], os.environ["AZURE_SUBSCRIPTION"])
kp = KQL(path, template="markdown/report-sentinel.md")
```

To run locally
```python
import os, json, pandas as pd
from azure_notebook_reporting import KQL, BlobPath
path = BlobPath("..")
kp = KQL(path, template="markdown/report-sentinel.md")
```

In [ ]:
import os, json, pandas as pd
from azure_notebook_reporting import KQL, BlobPath

path = BlobPath("..")

kp = KQL(path, template="markdown/report-sentinel.md")
kp.set_agency(os.environ["REPORT_AGENCY"], sample_agency=os.environ["REPORT_SAMPLE_AGENCY"], sample_only=False)

kp.init_report(
    background = "https://raw.githubusercontent.com/wagov/wasoc-notebook/main/notebooks/background.svg",
    entity = kp.agency_name,
    date = kp.today.strftime("%B %Y"),
    body = "#121212",
    links = "#084595",
    titles = "#CC5733",
    footer = "#808080", 
)

## Loading data to report on
Execute queries against external apis and gather some statistics

In [ ]:
kp.load_queries({
    "Incident Details": "siemhealth/incidentdetail.kql",
    "Alert Details": "siemhealth/alertdetail.kql",
    "Local Admins": "siemhealth/localadminlogons.kql",
    "Email Delivery": "siemhealth/emaildelivery.kql",
    "External Files": "siemhealth/externaldownloads.kql",
    "Guest Tenants": "siemhealth/guestdownloads.kql",
    "On Premise Logons": "siemhealth/identitylogonevents.kql",
    "Azure AD Logons": "siemhealth/signins.kql",
    "Operating Systems": "siemhealth/operatingsystems.kql",
    "Ingestion Detail": "siemhealth/usage.kql",
})

## Composing a report

In [ ]:
%matplotlib agg
section = "Executive Summary"

exec_summary = kp.nbpath / f"markdown/exec_summaries/{kp.agency}-{kp.today.strftime('%b%Y')}.md"
if not exec_summary.exists():
    exec_summary = kp.nbpath / "markdown/exec_summaries/default.md"

kp.report[section] = kp.report_sections[section].substitute(
    agency = kp.agency_name,
    date = kp.today.strftime("%B %Y"),
    summary = exec_summary.open().read()
)

kp.report += KQL.minitable(kp.querystats)

section = "Tactics and Rules"
kp.report[section] = kp.report_sections[section].substitute()

if kp.querystats["Rows"]["Incident Details"] == 0:
    kp.report[section] += "*MISSING DATA: Please confirm there are [analytics rules](https://learn.microsoft.com/en-us/azure/sentinel/detect-threats-built-in) configured on the Analytics page.*"
    kp.report[section] += f"** SAMPLE DATASET BELOW (substituting for {querystats['Columns']['Incident Details']} **"
df = kp.queries["Incident Details"].groupby(["Status", "Classification", "Severity", "Tactics", "Title"])
df = df["IncidentNumber"].nunique().unstack(level=[0,1]).dropna(axis=1, how="all").fillna(0)
df["Total"] = df.sum(numeric_only=True, axis=1)
df = df.sort_values("Total", ascending=False).replace({0: ""}).head(5)
kp.report[section] += KQL.minitable(df)

df, interval = KQL.dfago(kp.queries["Incident Details"], "7D"), "3H"
tactics = df.groupby("Tactics").resample(interval, on="TimeGenerated")["IncidentNumber"].nunique().reset_index()
kp.report[section] += KQL.df2fig(tactics, "Detections by Tactic", "TimeGenerated", "IncidentNumber", "Tactics")
rules = df.groupby("Title").resample(interval, on="TimeGenerated")["IncidentNumber"].nunique().reset_index().rename(columns={"Title": "Rule"})
kp.report[section] += KQL.df2fig(rules, "Detections by Rule", "TimeGenerated", "IncidentNumber", "Rule")

In [ ]:
section = "Cost Optimisation"
kp.report[section] = kp.report_sections[section].substitute()

df = KQL.dfago(kp.queries["Ingestion Detail"], "5D")
df["GB"] = df.IngestionVolume.map(lambda v: v / 1000)
kp.report[section] += KQL.df2fig(df[df["Billable"] == "True"], "Billable Ingestion (GB) by Table", "TimeGenerated", "GB", "Table")

In [ ]:
kp.report_pdf()